# 01 - EDA: Computer Marketplace Dataset

## Objective

This notebook performs an initial exploration of `db_computers_2025_raw.csv` - a dataset of computer listings from a marketplace.

### Goals

1. **Understand the target variable** (price) - its distribution, range, and potential transformations
2. **Inspect distributions and missing values** across all columns
3. **Identify core features** for the initial model:
   - Product type (Tipo de producto)
   - Brand (Marca) and Series (Serie)
   - Processor (Procesador_Procesador)
   - Graphics card (Gráfica_Tarjeta gráfica)
   - RAM, storage, screen specs
4. **Prepare observations** for the feature engineering notebook

### Important Conventions

- **Original Spanish column names are preserved** - we do NOT rename or translate columns
- All comments and markdown are in English for readability
- Engineered features (created in later notebooks) will start with underscore: `_cpu_mark`, `_ram_gb`, etc.

---

## 1. Imports and Display Options

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Seaborn style
sns.set_theme(style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 2. Load Data

In [ ]:
# Define paths (relative to notebooks/ folder)
DATA_DIR = Path('../data')

# Main dataset
computers_path = DATA_DIR / 'db_computers_2025_raw.csv'
columns_info_path = DATA_DIR / 'db_computers_columns_names.txt'

# Load the main dataset
# Note: Using utf-8-sig to handle BOM, index_col=0 then reset_index per course instructions
df = pd.read_csv(
    computers_path,
    encoding='utf-8-sig',
    index_col=0
).reset_index()

# Rename the old index column if needed
if 'index' in df.columns:
    df = df.rename(columns={'index': 'id_original'})

print(f"Dataset shape: {df.shape}")
print(f"  - Rows (computer listings): {df.shape[0]:,}")
print(f"  - Columns: {df.shape[1]}")

In [ ]:
# Preview the first few rows
# Note: Original Spanish column names are preserved exactly as they appear in the CSV
df.head()

In [ ]:
# List all columns to see what's available
print("All columns in the dataset:\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i:3d}. {col}")

## 3. Basic Info and Data Types

In [ ]:
# Get basic info about the dataset
df.info()

In [ ]:
# Check unique values for key categorical columns
# These are columns we suspect will be important for prediction

key_columns = [
    'Tipo de producto',           # Product type (laptop, desktop, etc.)
    'Serie',                       # Product series/line
    'Tipo',                        # Type (Laptop, etc.)
    'Procesador_Fabricante del procesador',  # CPU manufacturer (Intel, AMD, Apple)
    'Gráfica_Fabricante de la tarjeta gráfica',  # GPU manufacturer
    'Sistema operativo_Sistema operativo',  # OS
]

print("Unique value counts for key categorical columns:\n")
for col in key_columns:
    if col in df.columns:
        print(f"{col}:")
        print(f"  Unique values: {df[col].nunique()}")
        print(f"  Missing: {df[col].isna().sum()} ({df[col].isna().mean()*100:.1f}%)")
        print()

In [ ]:
# Value counts for product type - this is crucial for understanding our dataset
if 'Tipo de producto' in df.columns:
    print("Product types (Tipo de producto):\n")
    print(df['Tipo de producto'].value_counts(dropna=False))

In [ ]:
# Value counts for 'Tipo' column
if 'Tipo' in df.columns:
    print("Type (Tipo):\n")
    print(df['Tipo'].value_counts(dropna=False))

In [ ]:
# CPU manufacturers
if 'Procesador_Fabricante del procesador' in df.columns:
    print("CPU Manufacturers (Procesador_Fabricante del procesador):\n")
    print(df['Procesador_Fabricante del procesador'].value_counts(dropna=False))

## 4. Target Variable Exploration: Price (Precio)

The price information is stored in the `Precio_Rango` column as a string range (e.g., "1.026,53 € – 2.287,17 €").

For modeling, we'll need to extract a numeric value from this range. For now, let's explore it as-is.

In [ ]:
# Identify the price column
precio_col = 'Precio_Rango'

print(f"Price column: {precio_col}")
print(f"\nSample values:")
print(df[precio_col].head(10))

In [ ]:
def extraer_precio_numerico(precio_str):
    """
    Extract numeric price from a range string.
    For ranges like "1.026,53 € – 2.287,17 €", returns the midpoint.
    For single prices, returns that price.
    
    Spanish format: periods for thousands, commas for decimals.
    """
    if pd.isna(precio_str) or not isinstance(precio_str, str):
        return np.nan
    
    import re
    
    # Find all price patterns (number with Spanish format)
    # Pattern: digits with optional periods, comma, digits
    pattern = r'([\d.]+,\d{2})'
    matches = re.findall(pattern, precio_str)
    
    if not matches:
        return np.nan
    
    # Convert Spanish format to float
    precios = []
    for m in matches:
        # Remove thousand separators (.), replace decimal comma with period
        num_str = m.replace('.', '').replace(',', '.')
        precios.append(float(num_str))
    
    # Return midpoint if range, otherwise single value
    if len(precios) == 2:
        return (precios[0] + precios[1]) / 2
    elif len(precios) == 1:
        return precios[0]
    else:
        return np.mean(precios)

# Extract numeric prices (for EDA only - proper extraction in features.py)
df['_precio_eda'] = df[precio_col].apply(extraer_precio_numerico)

print(f"Extracted numeric prices:")
print(f"  Valid prices: {df['_precio_eda'].notna().sum()} / {len(df)}")
print(f"  Missing: {df['_precio_eda'].isna().sum()}")

In [ ]:
# Price statistics
print("Price Statistics (in €):\n")
print(df['_precio_eda'].describe())

In [ ]:
# Price distribution visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
df['_precio_eda'].hist(bins=50, ax=ax1, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Price (€)')
ax1.set_ylabel('Frequency')
ax1.set_title('Price Distribution (Histogram)')
ax1.axvline(df['_precio_eda'].median(), color='red', linestyle='--', label=f"Median: {df['_precio_eda'].median():,.0f}€")
ax1.axvline(df['_precio_eda'].mean(), color='orange', linestyle='--', label=f"Mean: {df['_precio_eda'].mean():,.0f}€")
ax1.legend()

# Box plot
ax2 = axes[1]
df.boxplot(column='_precio_eda', ax=ax2)
ax2.set_ylabel('Price (€)')
ax2.set_title('Price Distribution (Box Plot)')

plt.tight_layout()
plt.show()

# Observations on price distribution
print("\nObservations:")
print("- The distribution appears right-skewed (long tail of high prices)")
print("- There are potential outliers at the high end (expensive gaming/workstation PCs)")
print("- A log-transform might help normalize the distribution for modeling")

## 4.2. Outlier Analysis

Outliers in price data can significantly affect model training (especially RMSE). Let's identify and analyze potential outliers.

In [ ]:
# Outlier detection using multiple methods
print("=" * 60)
print("OUTLIER ANALYSIS FOR PRICE")
print("=" * 60)

price_data = df['_precio_eda'].dropna()

# Method 1: IQR (Interquartile Range)
Q1 = price_data.quantile(0.25)
Q3 = price_data.quantile(0.75)
IQR = Q3 - Q1
lower_bound_iqr = Q1 - 1.5 * IQR
upper_bound_iqr = Q3 + 1.5 * IQR

outliers_iqr = price_data[(price_data < lower_bound_iqr) | (price_data > upper_bound_iqr)]

print(f"\n1. IQR Method (1.5 * IQR):")
print(f"   Q1: €{Q1:,.0f}, Q3: €{Q3:,.0f}, IQR: €{IQR:,.0f}")
print(f"   Lower bound: €{lower_bound_iqr:,.0f}")
print(f"   Upper bound: €{upper_bound_iqr:,.0f}")
print(f"   Outliers detected: {len(outliers_iqr)} ({len(outliers_iqr)/len(price_data)*100:.1f}%)")

# Method 2: Percentile-based (1st and 99th percentile)
p1 = price_data.quantile(0.01)
p99 = price_data.quantile(0.99)
outliers_percentile = price_data[(price_data < p1) | (price_data > p99)]

print(f"\n2. Percentile Method (1st-99th):")
print(f"   1st percentile: €{p1:,.0f}")
print(f"   99th percentile: €{p99:,.0f}")
print(f"   Outliers detected: {len(outliers_percentile)} ({len(outliers_percentile)/len(price_data)*100:.1f}%)")

# Method 3: Z-score (values > 3 std from mean)
mean_price = price_data.mean()
std_price = price_data.std()
z_threshold = 3
lower_bound_z = mean_price - z_threshold * std_price
upper_bound_z = mean_price + z_threshold * std_price
outliers_zscore = price_data[(price_data < lower_bound_z) | (price_data > upper_bound_z)]

print(f"\n3. Z-Score Method (|z| > 3):")
print(f"   Mean: €{mean_price:,.0f}, Std: €{std_price:,.0f}")
print(f"   Lower bound: €{max(0, lower_bound_z):,.0f}")
print(f"   Upper bound: €{upper_bound_z:,.0f}")
print(f"   Outliers detected: {len(outliers_zscore)} ({len(outliers_zscore)/len(price_data)*100:.1f}%)")

# Show the extreme outliers
print(f"\n=== Most Extreme High Prices (Top 10) ===")
print(price_data.nlargest(10).to_string())

print(f"\n=== Most Extreme Low Prices (Bottom 10) ===")
print(price_data.nsmallest(10).to_string())

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Distribution with outlier thresholds
ax1 = axes[0]
df['_precio_eda'].hist(bins=100, ax=ax1, edgecolor='black', alpha=0.7)
ax1.axvline(upper_bound_iqr, color='red', linestyle='--', lw=2, label=f'IQR upper: €{upper_bound_iqr:,.0f}')
ax1.axvline(p99, color='orange', linestyle='--', lw=2, label=f'99th pct: €{p99:,.0f}')
ax1.set_xlabel('Price (€)')
ax1.set_ylabel('Frequency')
ax1.set_title('Price Distribution with Outlier Thresholds')
ax1.legend()

# 2. Box plot showing outliers
ax2 = axes[1]
ax2.boxplot(price_data, vert=True)
ax2.set_ylabel('Price (€)')
ax2.set_title('Box Plot (dots are outliers)')

# 3. Distribution WITHOUT outliers (preview)
ax3 = axes[2]
price_no_outliers = price_data[(price_data >= p1) & (price_data <= p99)]
price_no_outliers.hist(bins=50, ax=ax3, edgecolor='black', alpha=0.7, color='green')
ax3.set_xlabel('Price (€)')
ax3.set_ylabel('Frequency')
ax3.set_title(f'Price Distribution WITHOUT Outliers\n(1st-99th percentile, n={len(price_no_outliers):,})')

plt.tight_layout()
plt.show()

# Summary statistics comparison
print("\n=== Impact of Removing Outliers (1st-99th percentile) ===")
print(f"{'Metric':<15s} {'With Outliers':>15s} {'Without Outliers':>18s} {'Change':>12s}")
print("-" * 60)
print(f"{'Count':<15s} {len(price_data):>15,} {len(price_no_outliers):>18,} {len(price_data)-len(price_no_outliers):>+12,}")
print(f"{'Mean':<15s} €{price_data.mean():>14,.0f} €{price_no_outliers.mean():>17,.0f} {(price_no_outliers.mean()-price_data.mean()):>+12,.0f}")
print(f"{'Std':<15s} €{price_data.std():>14,.0f} €{price_no_outliers.std():>17,.0f} {(price_no_outliers.std()-price_data.std()):>+12,.0f}")
print(f"{'Min':<15s} €{price_data.min():>14,.0f} €{price_no_outliers.min():>17,.0f}")
print(f"{'Max':<15s} €{price_data.max():>14,.0f} €{price_no_outliers.max():>17,.0f}")

### Outlier Handling Recommendation

Based on the analysis above:

**Recommendation: Remove extreme outliers using the 1st-99th percentile method**

Reasons:
1. **RMSE sensitivity**: RMSE squares errors, so extreme outliers disproportionately affect training
2. **Data quality**: Very low prices (< €200) may be accessories or errors; very high prices (> €5000) are rare workstations
3. **Model generalization**: Most users want predictions for "typical" computers, not extreme cases
4. **Conservative approach**: 1st-99th percentile removes only ~2% of data while reducing noise

**Implementation in `features.py`:**
```python
# Remove price outliers (1st-99th percentile)
p1 = df['_precio_num'].quantile(0.01)
p99 = df['_precio_num'].quantile(0.99)
df = df[(df['_precio_num'] >= p1) & (df['_precio_num'] <= p99)]
```

**Alternative**: For production, consider keeping outliers but using:
- MAE instead of RMSE (less sensitive to outliers)
- Quantile regression (robust to outliers)
- Log-transform target (compresses extreme values)

In [ ]:
# Log-transformed price distribution (preview for future consideration)
fig, ax = plt.subplots(figsize=(10, 5))

df['_log_precio'] = np.log1p(df['_precio_eda'])
df['_log_precio'].hist(bins=50, ax=ax, edgecolor='black', alpha=0.7, color='green')
ax.set_xlabel('Log(Price + 1)')
ax.set_ylabel('Frequency')
ax.set_title('Log-Transformed Price Distribution')

plt.tight_layout()
plt.show()

print("Note: Log-transform makes the distribution more symmetric.")
print("Consider using log-price as target, then exp() to get final predictions.")

## 4.1. Price Range Analysis: Range vs Midpoint

The `Precio_Rango` column contains price ranges (min-max). We need to decide whether to:
1. **Predict the midpoint** (simpler, single target)
2. **Predict the range** (min and max separately, or range spread)

Let's analyze the range spreads to inform this decision.

In [ ]:
# Extract min, max, and midpoint from price ranges
import re

def extract_full_price_range(precio_str):
    """Extract min, max, and midpoint from price range string."""
    if pd.isna(precio_str) or not isinstance(precio_str, str):
        return None, None, None
    
    pattern = r'([\d.]+,\d{2})'
    matches = re.findall(pattern, precio_str)
    
    if not matches:
        return None, None, None
    
    # Convert Spanish format to float
    precios = []
    for m in matches:
        num_str = m.replace('.', '').replace(',', '.')
        precios.append(float(num_str))
    
    if len(precios) == 2:
        min_price = precios[0]
        max_price = precios[1]
        mid_price = (precios[0] + precios[1]) / 2
        return min_price, max_price, mid_price
    elif len(precios) == 1:
        return precios[0], precios[0], precios[0]
    else:
        return None, None, None

# Apply extraction
price_ranges = df['Precio_Rango'].apply(extract_full_price_range)
df['_min_price'] = price_ranges.apply(lambda x: x[0])
df['_max_price'] = price_ranges.apply(lambda x: x[1])
df['_mid_price'] = price_ranges.apply(lambda x: x[2])

# Calculate spread metrics
df['_price_spread'] = df['_max_price'] - df['_min_price']
df['_spread_pct'] = (df['_price_spread'] / df['_min_price'] * 100)

print("Price Range Statistics:\n")
print(df[['_min_price', '_max_price', '_mid_price', '_price_spread', '_spread_pct']].describe())

In [ ]:
# Visualize price spread distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Price spread in euros
ax1 = axes[0]
df['_price_spread'].hist(bins=50, ax=ax1, edgecolor='black', alpha=0.7, color='coral')
ax1.set_xlabel('Price Spread (€)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Price Range Spread (Max - Min)')
ax1.axvline(df['_price_spread'].median(), color='red', linestyle='--', 
            label=f"Median: {df['_price_spread'].median():,.0f}€")

# Price spread as percentage
ax2 = axes[1]
df['_spread_pct'].hist(bins=50, ax=ax2, edgecolor='black', alpha=0.7, color='skyblue')
ax2.set_xlabel('Spread as % of Min Price')
ax2.set_ylabel('Frequency')
ax2.set_title('Price Spread as Percentage of Min Price')
ax2.axvline(df['_spread_pct'].median(), color='red', linestyle='--', 
            label=f"Median: {df['_spread_pct'].median():.1f}%")

for ax in axes:
    ax.legend()

plt.tight_layout()
plt.show()

print("\nObservations:")
print(f"- Median price spread: {df['_price_spread'].median():,.0f}€")
print(f"- Median spread as % of min price: {df['_spread_pct'].median():.1f}%")
print(f"- Mean spread as % of min price: {df['_spread_pct'].mean():.1f}%")
print("\n-> The price ranges are quite wide (median ~{:.0f}% of min price)".format(df['_spread_pct'].median()))
print("-> This suggests the marketplace aggregates offers from multiple sellers")

In [ ]:
# Check Ofertas column - does it contain individual price listings?
print("=== Ofertas Column Analysis ===\n")
print("Sample values:")
print(df['Ofertas'].head(20))
print("\n")
print(f"Unique values: {df['Ofertas'].nunique()}")
print(f"Missing values: {df['Ofertas'].isna().sum()} ({df['Ofertas'].isna().mean()*100:.1f}%)")
print("\n")

# Extract number of offers
def extract_num_ofertas(oferta_str):
    """Extract number of offers from string like '200 ofertas:'"""
    if pd.isna(oferta_str):
        return np.nan
    import re
    match = re.search(r'(\d+)\s+ofertas?', str(oferta_str), re.IGNORECASE)
    if match:
        return int(match.group(1))
    return np.nan

df['_num_ofertas'] = df['Ofertas'].apply(extract_num_ofertas)

print("Number of offers statistics:")
print(df['_num_ofertas'].describe())
print("\n")

# Correlation between number of offers and price spread
correlation = df[['_num_ofertas', '_price_spread', '_spread_pct']].corr()
print("Correlation with price spread:")
print(correlation.loc['_num_ofertas'])
print("\n")

print("Conclusion:")
print("- The 'Ofertas' column only contains the COUNT of offers (e.g., '200 ofertas:')")
print("- It does NOT contain individual price listings")
print("- The Precio_Rango represents the min-max across all those offers")
print("- More offers tends to correlate with wider price spreads")

### Decision: Predict Midpoint vs Range?

Based on the analysis above, here's the recommendation:

**Recommendation: Predict the MIDPOINT price** for the following reasons:

1. **Simpler modeling approach**: Single target variable instead of two (min/max)
2. **Marketplace context**: The price range represents offers from multiple sellers, not product uncertainty
3. **User-friendly output**: A single price estimate is easier to interpret than a range
4. **Wide spreads**: The ranges are wide (~95% median spread), so predicting the exact range is less meaningful

**Alternative approaches** (for future consideration):
- Predict both `min_price` and `max_price` separately (multi-target regression)
- Predict `midpoint` + `spread` (could give users a confidence interval)
- Build separate models for different price segments

**Decision**: We'll use `_precio_num` (midpoint) as our target variable in `02_feature_engineering.ipynb`

## 5. Missing Values Overview

Understanding missing values is crucial for:
- Deciding which features to include in the model
- Choosing imputation strategies
- Identifying potentially unreliable columns

In [ ]:
# Calculate missing values per column
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)

# Create a summary DataFrame
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_pct,
    'Present %': 100 - missing_pct
})

# Show columns with most missing values
print("Columns with MOST missing values (top 30):\n")
print(missing_df.head(30))

In [ ]:
# Show columns with LEAST missing values (most complete)
print("Columns with LEAST missing values (most complete):\n")
print(missing_df.tail(30))

In [ ]:
# Visualize missing values for key feature columns
# Focus on columns we expect to use for modeling

key_feature_cols = [
    'Precio_Rango',
    'Título',
    'Tipo de producto',
    'Serie',
    'Procesador_Procesador',
    'Procesador_Fabricante del procesador',
    'Procesador_Número de núcleos del procesador',
    'RAM_Memoria RAM',
    'RAM_Tipo de RAM',
    'Disco duro_Capacidad de memoria SSD',
    'Disco duro_Tipo de disco duro',
    'Gráfica_Tarjeta gráfica',
    'Gráfica_Fabricante de la tarjeta gráfica',
    'Gráfica_Memoria gráfica',
    'Pantalla_Diagonal de la pantalla',
    'Pantalla_Resolución de pantalla',
    'Tipo',
]

# Filter to columns that exist
key_feature_cols = [c for c in key_feature_cols if c in df.columns]

key_missing = missing_df.loc[key_feature_cols].sort_values('Missing %', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
key_missing['Present %'].plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Data Completeness (%)')
ax.set_title('Data Completeness for Key Feature Columns')
ax.set_xlim(0, 100)

# Add percentage labels
for i, v in enumerate(key_missing['Present %']):
    ax.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Categorize columns by missing data levels
very_sparse = missing_df[missing_df['Missing %'] >= 80].index.tolist()  # >80% missing
sparse = missing_df[(missing_df['Missing %'] >= 50) & (missing_df['Missing %'] < 80)].index.tolist()
moderate = missing_df[(missing_df['Missing %'] >= 20) & (missing_df['Missing %'] < 50)].index.tolist()
dense = missing_df[missing_df['Missing %'] < 20].index.tolist()  # <20% missing

print(f"Very Sparse (>80% missing): {len(very_sparse)} columns")
print(f"Sparse (50-80% missing): {len(sparse)} columns")
print(f"Moderate (20-50% missing): {len(moderate)} columns")
print(f"Dense (<20% missing): {len(dense)} columns")

print("\n--- Dense columns (likely useful for initial model) ---")
for col in dense:
    print(f"  {col}: {missing_df.loc[col, 'Present %']:.1f}% present")

### Missing Values Observations

**Dense columns (low missingness, likely useful):**
- `Título` (Title) - Always present, can extract brand info
- `Precio_Rango` - Target variable, mostly present
- `Tipo de producto` - Product category (gaming, multimedia, etc.)
- `Tipo` - Laptop vs Desktop classification

**Moderate missingness (need imputation):**
- `Procesador_Procesador` - CPU name, will need fuzzy matching to benchmarks
- `RAM_Memoria RAM` - RAM info, extractable with regex
- Screen-related columns

**Sparse columns (might drop or use carefully):**
- Many detailed spec columns are very sparse
- Some columns only apply to specific product types (e.g., battery for laptops only)

## 6. First Thoughts on Core Features

Based on domain knowledge about computer pricing, here are the features we expect to be most predictive:

In [ ]:
# Core features we plan to use for the initial model
# Note: We'll create engineered versions of many of these with _ prefix

posibles_features_core = [
    # Category/Type
    'Tipo de producto',           # Product type (gaming, multimedia, professional)
    'Tipo',                        # Laptop vs Desktop
    'Serie',                       # Product series (often indicates tier)
    
    # Processor
    'Procesador_Procesador',       # CPU model - will map to _cpu_mark benchmark
    'Procesador_Fabricante del procesador',  # Intel, AMD, Apple
    'Procesador_Número de núcleos del procesador',  # Core count
    
    # Graphics
    'Gráfica_Tarjeta gráfica',     # GPU model - will map to _gpu_mark benchmark
    'Gráfica_Fabricante de la tarjeta gráfica',  # NVIDIA, AMD, Intel
    'Gráfica_Memoria gráfica',     # VRAM
    
    # Memory and Storage
    'RAM_Memoria RAM',             # RAM - will extract to _ram_gb
    'RAM_Tipo de RAM',             # DDR4, DDR5, LPDDR5X
    'Disco duro_Capacidad de memoria SSD',  # SSD - will extract to _ssd_gb
    'Disco duro_Tipo de disco duro',  # SSD, HDD, hybrid
    
    # Display
    'Pantalla_Diagonal de la pantalla',  # Screen size - will extract to _tamano_pantalla_pulgadas
    'Pantalla_Resolución de pantalla',   # Resolution
    'Pantalla_Tasa de actualización de imagen',  # Refresh rate (important for gaming)
]

print("Core features for initial model:")
for i, feat in enumerate(posibles_features_core, 1):
    if feat in df.columns:
        present_pct = (1 - df[feat].isna().mean()) * 100
        print(f"{i:2d}. {feat}: {present_pct:.1f}% present")
    else:
        print(f"{i:2d}. {feat}: NOT FOUND")

## 6.1. Brand Extraction from Título

The dataset may not have a `Marca` (brand) column, but we can extract it from the `Título` column.
Let's explore how to identify brands from product titles.

In [ ]:
# Check if Marca column exists
if 'Marca' in df.columns:
    print("Marca column EXISTS\n")
    print(df['Marca'].value_counts(dropna=False).head(20))
    print(f"\nMissing: {df['Marca'].isna().sum()} ({df['Marca'].isna().mean()*100:.1f}%)")
else:
    print("Marca column DOES NOT EXIST - we need to extract it from Título\n")
    
# Sample titles to understand the pattern
print("\n=== Sample Títulos ===")
print(df['Título'].head(30))

In [ ]:
# Extract brand from title (usually the first word)
def extract_brand(titulo):
    """Extract brand from product title (usually first word)."""
    if pd.isna(titulo):
        return np.nan
    
    # Common computer brands (case insensitive)
    common_brands = [
        'Apple', 'ASUS', 'Lenovo', 'HP', 'Dell', 'Acer', 'MSI', 
        'Samsung', 'Microsoft', 'Razer', 'Alienware', 'LG', 
        'Huawei', 'Xiaomi', 'GigaByte', 'Gigabyte', 'Toshiba',
        'Fujitsu', 'Medion', 'Sony', 'Vaio', 'Corsair', 'NZXT'
    ]
    
    # Get first word (usually the brand)
    first_word = str(titulo).split()[0] if titulo else ''
    
    # Check if first word matches a known brand (case insensitive)
    for brand in common_brands:
        if first_word.lower() == brand.lower():
            return brand
    
    # If not found in common brands, return the first word anyway
    # (we'll clean this up in feature engineering)
    return first_word if first_word else np.nan

df['_brand_extracted'] = df['Título'].apply(extract_brand)

print("Extracted Brand Distribution:\n")
print(df['_brand_extracted'].value_counts(dropna=False).head(20))
print(f"\nUnique brands: {df['_brand_extracted'].nunique()}")
print(f"Missing: {df['_brand_extracted'].isna().sum()}")

# Compare with Serie column to see if brand info is there
print("\n=== Serie Column (for comparison) ===")
if 'Serie' in df.columns:
    print(df['Serie'].value_counts(dropna=False).head(20))

## 6.2. Serie Missing Value Imputation from Título

The `Serie` column has many missing values. Let's explore if we can infer the Series from the product title.

In [ ]:
# Analyze Serie column
print("=== Serie Column Analysis ===\n")
if 'Serie' in df.columns:
    print(f"Total rows: {len(df)}")
    print(f"Serie present: {df['Serie'].notna().sum()} ({df['Serie'].notna().mean()*100:.1f}%)")
    print(f"Serie missing: {df['Serie'].isna().sum()} ({df['Serie'].isna().mean()*100:.1f}%)")
    print("\n")
    
    # Look at some examples where Serie is present
    print("Examples with Serie present:")
    sample_with_serie = df[df['Serie'].notna()][['Título', 'Serie', '_brand_extracted']].head(15)
    print(sample_with_serie.to_string())
    print("\n")
    
    # Look at some examples where Serie is missing
    print("Examples with Serie MISSING:")
    sample_without_serie = df[df['Serie'].isna()][['Título', 'Serie', '_brand_extracted']].head(15)
    print(sample_without_serie.to_string())

In [ ]:
# Extract potential series from title
def extract_series_from_title(titulo, brand):
    """
    Attempt to extract product series from title.
    Common patterns:
    - Apple: MacBook Air, MacBook Pro, iMac
    - ASUS: ROG, Zenbook, Vivobook, TUF Gaming
    - Lenovo: ThinkPad, IdeaPad, Legion, LOQ
    - HP: Pavilion, Envy, Omen, EliteBook
    - Dell: Inspiron, XPS, Alienware, Latitude
    """
    if pd.isna(titulo):
        return np.nan
    
    titulo_lower = str(titulo).lower()
    
    # Known series patterns by brand
    series_patterns = {
        'Apple': ['MacBook Air', 'MacBook Pro', 'iMac', 'Mac Mini', 'Mac Pro', 'Mac Studio'],
        'ASUS': ['ROG', 'Zenbook', 'Vivobook', 'TUF Gaming', 'Republic of Gamers', 
                 'ExpertBook', 'ProArt', 'StudioBook', 'Chromebook'],
        'Lenovo': ['ThinkPad', 'IdeaPad', 'Legion', 'LOQ', 'Yoga', 'ThinkBook'],
        'HP': ['Pavilion', 'Envy', 'Omen', 'EliteBook', 'ProBook', 'Spectre', 'ZBook'],
        'Dell': ['Inspiron', 'XPS', 'Alienware', 'Latitude', 'Precision', 'Vostro'],
        'MSI': ['Katana', 'Stealth', 'Raider', 'Cyborg', 'Prestige', 'Modern', 'Summit'],
        'Acer': ['Aspire', 'Swift', 'Nitro', 'Predator', 'TravelMate', 'ConceptD'],
        'Samsung': ['Galaxy Book'],
        'Microsoft': ['Surface'],
        'Gigabyte': ['Aero', 'Aorus'],
    }
    
    if pd.isna(brand) or brand not in series_patterns:
        return np.nan
    
    # Look for series keywords in title
    for series in series_patterns[brand]:
        if series.lower() in titulo_lower:
            return series
    
    return np.nan

df['_series_extracted'] = df.apply(
    lambda row: extract_series_from_title(row['Título'], row['_brand_extracted']), 
    axis=1
)

print("=== Series Extraction Results ===\n")
print(f"Series extracted: {df['_series_extracted'].notna().sum()}")
print(f"Could not extract: {df['_series_extracted'].isna().sum()}")
print("\n")

# Compare extracted series with existing Serie column
if 'Serie' in df.columns:
    print("Comparison: Existing Serie vs Extracted Series")
    comparison = pd.DataFrame({
        'Serie_exists': df['Serie'].notna(),
        'Serie_extracted': df['_series_extracted'].notna()
    })
    
    print("\nCrosstab:")
    print(pd.crosstab(comparison['Serie_exists'], comparison['Serie_extracted'], 
                      rownames=['Serie column present'], 
                      colnames=['Extracted from title']))
    
    # Show some examples where we filled missing Serie
    print("\n=== Examples where we filled missing Serie ===")
    filled = df[(df['Serie'].isna()) & (df['_series_extracted'].notna())]
    if len(filled) > 0:
        print(filled[['Título', 'Serie', '_series_extracted', '_brand_extracted']].head(10).to_string())
        print(f"\nTotal filled: {len(filled)} rows")

## 6.3. Screen Size: Pulgadas vs Centimeters

There are two screen size columns:
- `Pantalla_Tamaño de la pantalla` - in pulgadas (inches) like "15,6 pulgadas"
- `Pantalla_Diagonal de la pantalla` - in centimeters like "39,624 cm"

Let's check which one is more complete and if we need both.

In [ ]:
# Compare both screen size columns
screen_cols = {
    'Pantalla_Tamaño de la pantalla': 'pulgadas (inches)',
    'Pantalla_Diagonal de la pantalla': 'cm'
}

print("=== Screen Size Column Comparison ===\n")
for col, unit in screen_cols.items():
    if col in df.columns:
        present_count = df[col].notna().sum()
        present_pct = df[col].notna().mean() * 100
        print(f"{col} ({unit}):")
        print(f"  Present: {present_count} ({present_pct:.1f}%)")
        print(f"  Missing: {df[col].isna().sum()} ({df[col].isna().mean()*100:.1f}%)")
        print(f"  Sample values: {df[col].dropna().head(5).tolist()}")
        print()

# Check if they're redundant (one can be converted from the other)
if 'Pantalla_Tamaño de la pantalla' in df.columns and 'Pantalla_Diagonal de la pantalla' in df.columns:
    print("\n=== Checking correlation between pulgadas and cm ===")
    
    # Extract numeric values from both
    def extract_pulgadas(val):
        if pd.isna(val):
            return np.nan
        import re
        # Match patterns like "15,6 pulgadas" or "15.6 pulgadas"
        match = re.search(r'([\d,\.]+)', str(val))
        if match:
            num_str = match.group(1).replace(',', '.')
            return float(num_str)
        return np.nan
    
    def extract_cm(val):
        if pd.isna(val):
            return np.nan
        import re
        # Match patterns like "39,624 cm"
        match = re.search(r'([\d,\.]+)', str(val))
        if match:
            num_str = match.group(1).replace(',', '.')
            return float(num_str)
        return np.nan
    
    df['_pulgadas_num'] = df['Pantalla_Tamaño de la pantalla'].apply(extract_pulgadas)
    df['_cm_num'] = df['Pantalla_Diagonal de la pantalla'].apply(extract_cm)
    
    # Check correlation (1 inch = 2.54 cm)
    df['_cm_from_pulgadas'] = df['_pulgadas_num'] * 2.54
    
    # Compare for rows where both are present
    both_present = df[(df['_pulgadas_num'].notna()) & (df['_cm_num'].notna())]
    
    if len(both_present) > 0:
        print(f"\nRows with both values: {len(both_present)}")
        print("\nSample comparison:")
        sample = both_present[['Pantalla_Tamaño de la pantalla', 'Pantalla_Diagonal de la pantalla', 
                               '_pulgadas_num', '_cm_num', '_cm_from_pulgadas']].head(10)
        print(sample.to_string())
        
        # Check if they're consistent (allowing for rounding)
        diff = np.abs(both_present['_cm_num'] - both_present['_cm_from_pulgadas'])
        print(f"\nDifference between cm column and converted pulgadas:")
        print(f"  Mean diff: {diff.mean():.3f} cm")
        print(f"  Max diff: {diff.max():.3f} cm")
        
        consistent = (diff < 0.1).sum()
        print(f"\n  Consistent (diff < 0.1 cm): {consistent}/{len(both_present)} ({consistent/len(both_present)*100:.1f}%)")
    
    print("\n=== Coverage Analysis ===")
    coverage = pd.DataFrame({
        'Has pulgadas': df['_pulgadas_num'].notna(),
        'Has cm': df['_cm_num'].notna()
    })
    print(pd.crosstab(coverage['Has pulgadas'], coverage['Has cm'], 
                      rownames=['Has pulgadas'], colnames=['Has cm'], margins=True))

### Screen Size Recommendation

Based on the analysis above:

**Recommendation: Use `Pantalla_Tamaño de la pantalla` (pulgadas/inches) as the primary feature**

Reasons:
1. **Industry standard**: Screen sizes are typically marketed in inches (13", 15.6", 17", etc.)
2. **More intuitive**: Users and domain experts think in inches for laptops/monitors
3. **Likely better coverage**: The pulgadas column probably has similar or better coverage than cm
4. **Redundancy**: If both columns are present and consistent, we only need one
5. **Feature engineering**: We can convert cm to inches if pulgadas is missing

**Strategy for feature engineering** ([02_feature_engineering.ipynb](notebooks/02_feature_engineering.ipynb)):
- Extract numeric value from `Pantalla_Tamaño de la pantalla` to create `_tamano_pantalla_pulgadas`
- If missing, convert `Pantalla_Diagonal de la pantalla` (cm ÷ 2.54 = inches)
- This gives us maximum coverage with a single, interpretable feature

In [ ]:
# Explore RAM column
print("RAM_Memoria RAM - Value distribution:\n")
if 'RAM_Memoria RAM' in df.columns:
    print(df['RAM_Memoria RAM'].value_counts(dropna=False).head(20))
    print("\n-> RAM seems mostly present, values like '16 GB RAM', '8 GB RAM', etc.")
    print("-> Will need to extract numeric value to _ram_gb")

In [ ]:
# Explore processor column
print("Procesador_Procesador - Sample values:\n")
if 'Procesador_Procesador' in df.columns:
    print(df['Procesador_Procesador'].value_counts(dropna=False).head(20))
    print("\n-> Many unique CPU models (Intel Core i7-13700H, AMD Ryzen 7, Apple M3, etc.)")
    print("-> Will need to map to benchmark scores using db_cpu_raw.csv and fuzzy matching")

In [ ]:
# Explore GPU column
print("Gráfica_Tarjeta gráfica - Sample values:\n")
if 'Gráfica_Tarjeta gráfica' in df.columns:
    print(df['Gráfica_Tarjeta gráfica'].value_counts(dropna=False).head(20))
    print("\n-> Some GPU fields are sparse (integrated graphics often not listed)")
    print("-> Will need fuzzy matching to db_gpu_raw.csv for _gpu_mark")

In [ ]:
# Explore SSD capacity column
print("Disco duro_Capacidad de memoria SSD - Sample values:\n")
if 'Disco duro_Capacidad de memoria SSD' in df.columns:
    print(df['Disco duro_Capacidad de memoria SSD'].value_counts(dropna=False).head(20))
    print("\n-> Values like '512 GB', '1.000 GB', '256 GB'")
    print("-> Will extract to numeric _ssd_gb")

In [ ]:
# Explore screen size column
print("Pantalla_Diagonal de la pantalla - Sample values:\n")
if 'Pantalla_Diagonal de la pantalla' in df.columns:
    print(df['Pantalla_Diagonal de la pantalla'].value_counts(dropna=False).head(15))
    print("\n-> Values in cm like '39,624 cm', '35,56 cm'")
    print("-> Will convert to inches for _tamano_pantalla_pulgadas")

In [ ]:
# Price by product type - initial exploration
if 'Tipo de producto' in df.columns:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Filter to types with enough samples
    tipo_counts = df['Tipo de producto'].value_counts()
    main_tipos = tipo_counts[tipo_counts >= 10].index
    
    df_plot = df[df['Tipo de producto'].isin(main_tipos)]
    
    df_plot.boxplot(column='_precio_eda', by='Tipo de producto', ax=ax)
    ax.set_xlabel('Tipo de producto')
    ax.set_ylabel('Price (€)')
    ax.set_title('Price Distribution by Product Type')
    plt.suptitle('')  # Remove automatic title
    plt.xticks(rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    print("\nObservation: Gaming laptops (Portátil gaming) tend to have higher prices.")
    print("Product type will be an important categorical feature.")

## 7. Correlation Analysis: Numerical Features vs Price

Extract all numerical features and analyze their correlation with price.

In [ ]:
# Extract numerical features for correlation analysis
import re

# Extract RAM (GB)
def extract_numeric_gb(text):
    """Extract numeric value with GB/TB units."""
    if pd.isna(text):
        return np.nan
    text = str(text).upper().replace('.', '')  # Remove thousand separators
    # Handle TB
    match_tb = re.search(r'(\d+)\s*TB', text)
    if match_tb:
        return float(match_tb.group(1)) * 1000
    # Handle GB
    match_gb = re.search(r'(\d+)\s*GB', text)
    if match_gb:
        return float(match_gb.group(1))
    return np.nan

# Extract general numeric values (improved)
def extract_numeric(text):
    """Extract first numeric value from text."""
    if pd.isna(text):
        return np.nan
    # Look for integers or decimals (with comma or period as decimal separator)
    match = re.search(r'(\d+(?:[.,]\d+)?)', str(text))
    if match:
        num_str = match.group(1).replace(',', '.')
        try:
            return float(num_str)
        except ValueError:
            return np.nan
    return np.nan

# Extract resolution components
def extract_resolution(res_str):
    """Extract width and height from resolution."""
    if pd.isna(res_str):
        return np.nan, np.nan
    match = re.search(r'(\d+)\s*[xX×]\s*(\d+)', str(res_str))
    if match:
        return float(match.group(1)), float(match.group(2))
    return np.nan, np.nan

# Feature extraction
df['_ram_gb'] = df['RAM_Memoria RAM'].apply(extract_numeric_gb) if 'RAM_Memoria RAM' in df.columns else np.nan
df['_ssd_gb'] = df['Disco duro_Capacidad de memoria SSD'].apply(extract_numeric_gb) if 'Disco duro_Capacidad de memoria SSD' in df.columns else np.nan
df['_screen_inches'] = df['Pantalla_Tamaño de la pantalla'].apply(extract_numeric) if 'Pantalla_Tamaño de la pantalla' in df.columns else np.nan
df['_cpu_cores'] = df['Procesador_Número de núcleos del procesador'].apply(extract_numeric) if 'Procesador_Número de núcleos del procesador' in df.columns else np.nan
df['_gpu_memory_gb'] = df['Gráfica_Memoria gráfica'].apply(extract_numeric_gb) if 'Gráfica_Memoria gráfica' in df.columns else np.nan

# Resolution
if 'Pantalla_Resolución de pantalla' in df.columns:
    res_data = df['Pantalla_Resolución de pantalla'].apply(extract_resolution)
    df['_res_width'] = res_data.apply(lambda x: x[0])
    df['_res_height'] = res_data.apply(lambda x: x[1])
    df['_total_pixels'] = df['_res_width'] * df['_res_height'] / 1_000_000  # In millions
else:
    df['_res_width'] = np.nan
    df['_res_height'] = np.nan
    df['_total_pixels'] = np.nan

# Refresh rate
if 'Pantalla_Tasa de actualización de imagen' in df.columns:
    df['_refresh_rate'] = df['Pantalla_Tasa de actualización de imagen'].apply(extract_numeric)
else:
    df['_refresh_rate'] = np.nan

# Weight (convert to kg)
if 'Dimensiones_Peso' in df.columns:
    def extract_weight_kg(text):
        if pd.isna(text):
            return np.nan
        text = str(text).replace(',', '.')
        match_kg = re.search(r'([\d\.]+)\s*kg', text, re.IGNORECASE)
        if match_kg:
            try:
                return float(match_kg.group(1))
            except ValueError:
                return np.nan
        match_g = re.search(r'([\d\.]+)\s*g', text, re.IGNORECASE)
        if match_g:
            try:
                return float(match_g.group(1)) / 1000
            except ValueError:
                return np.nan
        return np.nan
    df['_weight_kg'] = df['Dimensiones_Peso'].apply(extract_weight_kg)
else:
    df['_weight_kg'] = np.nan

print("Extracted numerical features:")
features = ['_ram_gb', '_ssd_gb', '_screen_inches', '_cpu_cores', '_gpu_memory_gb', 
            '_total_pixels', '_refresh_rate', '_weight_kg', '_num_ofertas']
for feat in features:
    if feat in df.columns:
        valid = df[feat].notna().sum()
        pct = (valid / len(df)) * 100
        print(f"  {feat:20s}: {valid:5,} values ({pct:5.1f}%)")

In [ ]:
# Calculate correlations with mid price
features_to_correlate = ['_ram_gb', '_ssd_gb', '_screen_inches', '_cpu_cores', 
                         '_gpu_memory_gb', '_total_pixels', '_refresh_rate', 
                         '_weight_kg', '_num_ofertas']

# Filter to available features
available_features = [f for f in features_to_correlate if f in df.columns and df[f].notna().sum() > 10]

# Compute correlations
correlations = df[available_features + ['_mid_price']].corr()['_mid_price'].drop('_mid_price').sort_values(ascending=False)

print("\n=== Correlations with Price ===\n")
print(f"{'Feature':<20s} {'Correlation':>12s} {'Strength':>15s}")
print("-" * 50)
for feat, corr in correlations.items():
    if abs(corr) >= 0.5:
        strength = "Very Strong"
    elif abs(corr) >= 0.3:
        strength = "Moderate"
    else:
        strength = "Weak"
    print(f"{feat:<20s} {corr:>12.3f} {strength:>15s}")

# Visualization: Bar chart
fig, ax = plt.subplots(figsize=(10, 6))
correlations.plot(kind='barh', ax=ax, color=['green' if x > 0 else 'red' for x in correlations])
ax.set_xlabel('Correlation with Price', fontsize=12)
ax.set_title('Feature Correlations with Mid Price', fontsize=14, fontweight='bold')
ax.axvline(x=0.3, color='orange', linestyle='--', alpha=0.5, label='Moderate (0.3)')
ax.axvline(x=-0.3, color='orange', linestyle='--', alpha=0.5)
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Strong (0.5)')
ax.axvline(x=-0.5, color='red', linestyle='--', alpha=0.5)
ax.legend()
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

**Key Insights:**
- Strong correlations (|r| > 0.5) indicate features highly predictive of price
- Moderate correlations (0.3 < |r| < 0.5) are useful but less dominant
- Weak correlations (|r| < 0.3) may still be useful in combination with other features
- Negative correlations mean the feature inversely relates to price

## 8. Summary and Next Steps

The EDA revealed key insights about the computer marketplace dataset and informed our modeling strategy.

**Next:** Feature engineering in [02_feature_engineering.ipynb](02_feature_engineering.ipynb)

In [ ]:
# Cleanup temporary columns
temp_cols = ['_precio_eda', '_log_precio', '_min_price', '_max_price', '_mid_price', 
             '_price_spread', '_spread_pct', '_num_ofertas', '_brand_extracted', 
             '_series_extracted', '_pulgadas_num', '_cm_num', '_cm_from_pulgadas',
             '_ram_gb', '_ssd_gb', '_screen_inches', '_cpu_cores', '_gpu_memory_gb',
             '_res_width', '_res_height', '_total_pixels', '_refresh_rate', '_weight_kg']

dropped = 0
for col in temp_cols:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)
        dropped += 1

print(f"EDA complete! Dropped {dropped} temporary columns.")
print("Next: 02_feature_engineering.ipynb")